In [78]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math
import urllib
import random

In [2]:
def reset_tf():
    global sess
    sess.close()
    tf.reset_default_graph()
    sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))

In [3]:
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))

In [164]:
reset_tf()

n_h = 64
n_steps = 20
n_x = 256

X_raw = tf.placeholder(tf.int32, [None, n_steps], name='X_raw')
Y_raw = tf.placeholder(tf.int32, [None, n_steps], name='Y_raw')
H_init = tf.placeholder(tf.float32, [None, n_h], name='H_init')

W_xh = tf.get_variable("W_xh", shape=[n_x, n_h])
W_hh = tf.get_variable("W_hh", shape=[n_h, n_h])
W_ha = tf.get_variable("W_ha", shape=[n_h, n_x])
b_h = tf.get_variable("b_h", shape=[1, n_h])
b_a = tf.get_variable("b_a", shape=[1, n_x])

X   = [None] + tf.unstack(tf.one_hot(X_raw, n_x), axis=1)
Y   = [None] + tf.unstack(tf.one_hot(Y_raw, n_x), axis=1)
H   = [H_init] + [None] * n_steps
Z_h = [None] * (n_steps+1)
Z_a = [None] * (n_steps+1)
A   = [None] * (n_steps+1)

for i in range(1, n_steps + 1):
    Z_h[i] = tf.matmul(X[i], W_xh) + tf.matmul(H[i - 1], W_hh) + b_h
    H[i]   = tf.tanh(Z_h[i])
    Z_a[i] = tf.matmul(H[i], W_ha) + b_a
    A[i]   = tf.nn.softmax(Z_a[i])
    
loss = tf.constant(0.0)
for i in range(1, n_steps):
    loss += tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(labels=Y[i], logits=Z_a[i]))
loss /= tf.cast(tf.size(X_raw), tf.float32)

train_step = tf.train.AdamOptimizer(0.01).minimize(loss)

In [118]:
def next_batch(gens, n_steps):
    return np.array([[next(gen) for _ in range(n_steps)] for gen in gens])
    
def generate_repeating(n_x, offset = 0):
    i = offset
    while True:
        yield i % n_x
        i += 1

def generate_repeating_from_text(text, offset = 0):
    i = offset
    while True:
        yield ord(text[i % len(text)])
        i += 1

In [165]:
offsets = [random.randrange(len(text)) for _ in range(64)]
X_gens = [generate_repeating_from_text(text, offset) for offset in offsets]
Y_gens = [generate_repeating_from_text(text, offset + 1) for offset in offsets]

H_curr = np.random.randn(len(X_gens), n_h)

sess.run(tf.global_variables_initializer())

In [166]:
n_iters = 50000

for i in range(n_iters):
    X_batch = next_batch(X_gens, n_steps)
    Y_batch = next_batch(Y_gens, n_steps)
    feed_dict = {
        X_raw: X_batch,
        Y_raw: Y_batch,
        H_init: H_curr
    }
    sess.run(train_step, feed_dict)
    H_curr = sess.run(H[-1], feed_dict)
    if i % 100 == 0:
        print(f'\riteration: {i}, loss: {sess.run(loss, feed_dict)}',)

iteration: 0, loss: 4.781926155090332
iteration: 100, loss: 2.099501132965088
iteration: 200, loss: 1.8674484491348267
iteration: 300, loss: 1.6519635915756226
iteration: 400, loss: 1.5003186464309692
iteration: 500, loss: 1.4926798343658447
iteration: 600, loss: 1.3521798849105835
iteration: 700, loss: 1.2380061149597168
iteration: 800, loss: 1.2008930444717407
iteration: 900, loss: 1.131629228591919
iteration: 1000, loss: 1.0302391052246094
iteration: 1100, loss: 1.0959913730621338
iteration: 1200, loss: 0.9784004092216492
iteration: 1300, loss: 0.9142093658447266
iteration: 1400, loss: 0.9143632650375366
iteration: 1500, loss: 0.8206866979598999
iteration: 1600, loss: 0.8921475410461426
iteration: 1700, loss: 0.7915443778038025
iteration: 1800, loss: 0.7742353677749634
iteration: 1900, loss: 0.787238597869873
iteration: 2000, loss: 0.8067941665649414
iteration: 2100, loss: 0.8449389338493347
iteration: 2200, loss: 0.7789803147315979
iteration: 2300, loss: 0.7443720102310181
iteratio

iteration: 19400, loss: 0.5876443386077881
iteration: 19500, loss: 0.44702666997909546
iteration: 19600, loss: 0.5731972455978394
iteration: 19700, loss: 0.6051748991012573
iteration: 19800, loss: 0.4877173900604248
iteration: 19900, loss: 0.4617454409599304
iteration: 20000, loss: 0.6603344678878784
iteration: 20100, loss: 0.5008080005645752
iteration: 20200, loss: 0.4053410589694977
iteration: 20300, loss: 0.5765550136566162
iteration: 20400, loss: 0.5283342003822327
iteration: 20500, loss: 0.6232984662055969
iteration: 20600, loss: 0.450822651386261
iteration: 20700, loss: 0.4552507996559143
iteration: 20800, loss: 0.5929408669471741
iteration: 20900, loss: 0.5004450082778931
iteration: 21000, loss: 0.5066055059432983
iteration: 21100, loss: 0.48326197266578674
iteration: 21200, loss: 0.39515435695648193
iteration: 21300, loss: 0.48623543977737427
iteration: 21400, loss: 0.664553165435791
iteration: 21500, loss: 0.6491064429283142
iteration: 21600, loss: 0.4922238290309906
iteration

iteration: 38400, loss: 0.5467836856842041
iteration: 38500, loss: 0.48591312766075134
iteration: 38600, loss: 0.42390093207359314
iteration: 38700, loss: 1.1557247638702393
iteration: 38800, loss: 0.6767517924308777
iteration: 38900, loss: 0.6495878100395203
iteration: 39000, loss: 0.5635347962379456
iteration: 39100, loss: 0.3988146185874939
iteration: 39200, loss: 0.45693239569664
iteration: 39300, loss: 0.5794999003410339
iteration: 39400, loss: 0.5743770003318787
iteration: 39500, loss: 0.5441269874572754
iteration: 39600, loss: 0.6777375936508179
iteration: 39700, loss: 0.530519425868988
iteration: 39800, loss: 0.6580115556716919
iteration: 39900, loss: 0.5708794593811035
iteration: 40000, loss: 0.45716363191604614
iteration: 40100, loss: 0.5350274443626404
iteration: 40200, loss: 1.4171885251998901
iteration: 40300, loss: 1.049281120300293
iteration: 40400, loss: 0.9267711639404297
iteration: 40500, loss: 0.8382158279418945
iteration: 40600, loss: 0.7350532412528992
iteration: 4

KeyboardInterrupt: 

In [125]:
def sample(x_curr, H_curr):
    feed_dict = {
        X_raw: [[x_curr] * n_steps],
        H_init: H_curr.reshape(1, n_h)
    }
    H_new, p = sess.run((H[1], A[1][0]), feed_dict)
    x_new = np.random.choice(len(p), p = p)
    return x_new, H_new

def generate_samples(x_curr, H_curr):
    while True:
        yield x_curr
        x_curr, H_curr = sample(x_curr, H_curr)

In [168]:
g = generate_samples(0, H_curr[0])

''.join([chr(next(g)) for _ in range(100)])

'\x00ct. \r\n\r\n    Mr. Rabbit heard there. Mas?!hoodes, rale, batily wanked nhedly digging, he begancerne.'

In [143]:
text = None
with urllib.request.urlopen('http://textfiles.com/stories/13chil.txt') as response:
    text = response.read().decode("utf-8")

In [115]:
text = """the sun did not shine.
it was too wet to play.
so we sat in the house
all that cold, cold, wet day.

i sat there with sally.
we sat there, we two.
and i said, 'how i wish
we had something to do!'

too wet to go out
and too cold to play ball.
so we sat in the house.
we did nothing at all."""

"the sun did not shine.\nit was too wet to play.\nso we sat in the house\nall that cold, cold, wet day.\n\ni sat there with sally.\nwe sat there, we two.\nand i said, 'how i wish\nwe had something to do!'\n\ntoo wet to go out\nand too cold to play ball.\nso we sat in the house.\nwe did nothing at all."

In [145]:
len(text)

8457